In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ayush07120","key":"1d061f9217256b26d034f09cb5a00509"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images -p /content --unzip

Dataset URL: https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
License(s): ODbL-1.0
  0% 0.00/119M [00:00<?, ?B/s]
100% 119M/119M [00:00<00:00, 1.38GB/s]


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 224
BATCH_SIZE = 16

# Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    validation_split=0.18
)

In [22]:
train_dir = '/content/Data/train'  # Adjust if folder name is different

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Found 505 images belonging to 4 classes.
Found 108 images belonging to 4 classes.


In [75]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_57 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_55 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_56 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_57 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [76]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=45,
    validation_data=val_generator
)


Epoch 1/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 14s 365ms/step - accuracy: 0.3223 - loss: 1.7076 - val_accuracy: 0.5648 - val_loss: 1.1423
Epoch 2/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 300ms/step - accuracy: 0.4668 - loss: 1.1535 - val_accuracy: 0.5648 - val_loss: 0.9866
Epoch 3/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 310ms/step - accuracy: 0.5834 - loss: 1.0327 - val_accuracy: 0.5278 - val_loss: 0.9591
Epoch 4/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 318ms/step - accuracy: 0.5392 - loss: 0.9974 - val_accuracy: 0.6389 - val_loss: 0.7303
Epoch 5/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 318ms/step - accuracy: 0.5713 - loss: 0.9470 - val_accuracy: 0.6759 - val_loss: 0.8080
Epoch 6/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 318ms/step - accuracy: 0.5981 - loss: 0.9327 - val_accuracy: 0.6759 - val_loss: 0.8160
Epoch 7/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 323ms/step - accuracy: 0.6187 - loss: 0.8889 - val_accuracy: 0.6296 - val_loss: 0.8020
Epoch 8/45
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 323ms/step - accuracy: 0.6175 - loss: 0.8902 - val_accu

In [77]:
model.save("chest-cancer-updated-5.h5")

In [41]:
import tensorflow

In [78]:
# Load the pre-trained model: use your actual filename
model = tensorflow.keras.models.load_model('/content/chest-cancer-updated-5.h5')

In [79]:
import numpy as np
from tensorflow.keras.preprocessing import image

class_names = list(train_generator.class_indices.keys())

In [80]:
class_names

['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib',
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa',
 'normal',
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']

In [88]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img) / 255.0
    pred = model.predict(np.expand_dims(img_array, axis=0))
    i = np.argmax(pred[0])
    print(f"Predicted: {class_names[i]} ({pred[0][i]*100:.2f}%)")

# Example:
predict_image('/content/Data/test/adenocarcinoma/000119 (4).png')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted: squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa (42.61%)
